# Inheritance: For Better or for Worse
We'll cover
- the `super()` function
- the pitfalls of subclassing from built-in types
- Multiple inheritance and method resolution order
- Mixin classes

Discuss: Why is there backlash against the overuse of inheritance? What about multiple inheritance?

## The `super()` Function

In [57]:
import http.server
import socketserver
# from the collections module documentation

from collections import OrderedDict

class LastUpdatedOrderedDict(OrderedDict):
    """Store items in the order they were last updated"""

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.move_to_end(key)

In [58]:
# What about code that doesn't use `super()`, but instead calls the method directly on the superclass?

class NotRecommended(OrderedDict):
    """This is a counter example!"""

    def __setitem__(self, key, value):
        OrderedDict.__setitem__(self, key, value)
        self.move_to_end(key)

In [59]:
# You may see it like this, which is compatible in python 3 (and python 3)

class LastUpdatedOrderedDict(OrderedDict):
    """This code works in python 2 and 3"""

    def __setitem__(self, key, value):
        super(LastUpdatedOrderedDict, self).__setitem__(key, value)
        self.move_to_end(key)

## Subclassing Built-In Types is Tricky

Built-in types, like `list` or `dict` are written in C. This causes problems when you inherit directly from the
built-in type.

Example 14-1: Our `__setitem__` override is ignored by the `__init__` and `__update__` methods of the built-in dict

In [60]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [61]:
dd = DoppelDict(one=1)

In [62]:
dd

{'one': 1}

In [63]:
dd["two"] = 2

In [64]:
dd

{'one': 1, 'two': [2, 2]}

In [65]:
dd.update(three=3)

In [66]:
dd

{'one': 1, 'two': [2, 2], 'three': 3}

Example 14-2: The `__getitem__` of `AnswerDict` is bypassed by `dict.update`

In [67]:
class AnswerDict(dict):
    def __getitem__(self, item):
        return 42

In [68]:
ad = AnswerDict(a="foo")
print(ad)

{'a': 'foo'}


In [69]:
ad["a"]

42

In [70]:
ad.get("a")

'foo'

In [71]:
d = {}

In [72]:
d.update(ad)

In [73]:
d

{'a': 'foo'}

The solution? `collections.UserDict`

Example 14-3. `DoppelDict2` and `AnswerDict2` work as expected because they extend `UserDict` and not `dict`

In [74]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [75]:
dd = DoppelDict2(one=1)

In [76]:
dd

{'one': [1, 1]}

In [77]:
dd["two"] = 2

In [78]:
dd

{'one': [1, 1], 'two': [2, 2]}

In [79]:
dd.update(three=3)

In [80]:
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [81]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, item):
        return 42

In [82]:
ad = AnswerDict2(a="foo")
ad

{'a': 'foo'}

In [83]:
ad["a"]

42

In [84]:
d = {}

In [85]:
d.update(ad)

In [86]:
d["a"]

42

In [87]:
d

{'a': 42}

## Multiple Inheritance and Method Resolution Order

If a class has two superclasses, how does Python decide which attribute to use when we call `super().attr` when both
superclasses have an attribute with that name?

This is called the diamond problem.

Example 14-4. `diamond.py`: classes `Leaf`, `A`, `B`, `Root` form the graph.


In [88]:

class Root:  # <1>
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  # <3>
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  # <4>
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

Example 14-5. Doctests for calling ping and pong on a `Leaf` object

In [89]:
leaf1 = Leaf()

In [90]:
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [91]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In [92]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

Example 14-6: `diamond2.py`: classes to demonstrate the dynamic nature of `super()`

In [93]:
class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()

class U():  # <2>
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()  # <3>

class LeafUA(U, A):  # <4>
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()
# end::DIAMOND_CLASSES[]

class LeafAU(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafAU')
        super().ping()

In [94]:
u = U()

In [95]:
u.ping()

<__main__.U object at 0x7ff6844866b0>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [97]:
leaf2 = LeafUA()

In [98]:
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


In [99]:
LeafUA.__mro__

(__main__.LeafUA, __main__.U, __main__.A, __main__.Root, object)

In [100]:
leaf3 = LeafAU()
leaf3.ping()

<instance of LeafAU>.ping() in LeafAU
<instance of LeafAU>.ping() in A
<instance of LeafAU>.ping() in Root


In [101]:
LeafAU.__mro__

(__main__.LeafAU, __main__.A, __main__.Root, __main__.U, object)

Example 14-7. MRO of `tkinter.Text`

In [102]:
def print_mro(cls):
    print(", ".join(c.__name__ for c in cls.__mro__))

import tkinter
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


## Mixin Classes

A mixin class is designed to be subclassed together with at least one other class in a multiple inheritance arrangement.

A mixin is not supposed to be the only base class of a concrete class, because it does not provide all the
functionality for a concrete object, but only adds or customizes the behavior of child or sibling classes.

### Case-Insensitive Mappings

Example 14-8. `uppermixin.py`: `UpperCaseMixin` supports case-insensitive mappings

In [ ]:
import collections

def _upper(key):  # <1>
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:  # <2>
    """
    A class designed to provide case-insensitive access to mappings with string keys
    by uppercasing those keys when they are added or looked up
    """
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

Example 14-9. `uppermixin.py`: two classes that use `UpperCaseMixin`

In [103]:
class UpperDict(UpperCaseMixin, collections.UserDict):  # <1>
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):  # <2>
    """Specialized 'Counter' that uppercases string keys"""  # <3>

A quick demonstration of `UpperDict`

In [104]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])

In [105]:
list(d.keys())

['A', 2]

In [106]:
d['b'] = 'letter B'

In [107]:
d.keys()

KeysView({'A': 'letter A', 2: 'digit two', 'B': 'letter B'})

In [108]:
'b' in d

True

In [109]:
d['a'], d.get('B')

('letter A', 'letter B')

In [110]:
list(d.keys())

['A', 2, 'B']

A quick demonstration of `UpperCounter`

In [111]:
c = UpperCounter('BaNanA')
c.most_common()

[('A', 3), ('N', 2), ('B', 1)]

## Multiple inheritance in the Real World

### ABCs Are Mixins Too

### ThreadingMixIn and ForkingMixIn

The `http.server` package provides `HTTPServer` and `ThreadingHTTPServer` classes. The only difference is:

the threading one uses threads to handle requests by using the `ThreadingMixin`. This is useful to handle web
browsers pre-opening sockets, on which `HTTPServer` would wait indefinitely.

The complete source code for `ThreadingHTTPServer`:

In [ ]:

class ThreadingHTTPServer(socketserver.ThreadingMixIn, HTTPServer):
    daemon_threads = True

Example 14-10. Part of `Lib/sockerserver.py` in python 3.10

In [ ]:
class ThreadingMixIn:
    """Mixin class to handle each request in a new thread."""

    # 8 lines omitted

    def process_request_thread(selfself, request, client_address):
        ... # 6 lines omitted

    def process_request(self, request, client_address):
        ... # 8 lines omitted

    def server_close(self):
        super().server_close()
        self._threads.join()

## Django Generic Views Mixins

See book highlights

## Multiple Inheritance in Tkinter

See book highlights

In [112]:
import tkinter

print_mro(tkinter.Toplevel)

Toplevel, BaseWidget, Misc, Wm, object


In [113]:
print_mro(tkinter.Widget)

Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [114]:
print_mro(tkinter.Button)

Button, Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [115]:
print_mro(tkinter.Entry)

Entry, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, object


In [116]:
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


## Coping with Inheritance

### favor object composition over class inheritance

In [ ]:
class GeometryManager():
    def some_method(self):
        print('managing geometry')

In [ ]:
# inheritance
class Widget(GeometryManager):
    def widget_method(self):
        print("making widget")

In [ ]:
# composition
class Widget:
    def __init__(self):
        self.geometry_manager = GeometryManager()

### Understand Why Inheritance is Used in Each Case

The main reasons are:
 - Inheritance of interface creates a subtype, implying an "is-a" relationship. This is best done with ABCs.
 - Inheritance of implementation avoids code duplication by reuse. Mixins can help with this.

### Make Interfaces Explicit with ABCs

### Use Explicit Mixins for Code Reuse

### Provide Aggregate Classes to

From Django's source code, we have:


In [ ]:
class ListView(MultiipleObjectTemplateResponseMixin, BaseListView):
    """
    Render some list of objects, set by `self.model` or `self.queryset`.
    `self.queryset` can actually be an iterable of items, not just a queryset.
    """

Even though the body is empty, the class provides a useful sercice: it brings together a mixin and a base class that
should be used together.

## Subclass Only Classes Designed for Subclassing

Advice: Only create subclasses of classes that were designed to be subclassed.

How do you know if a class was designed to be subclassed?
- documentation
- `@final` - IDEs or type checkers can report misguided attempts to subclass those classes or override those methods

## Avoid Subclassing from Concrete Classes

Why is it dangerous to subclass concrete classes?

All non-leaf classes should be abstract. Said differently, only abstract classes should be subclassed.

# Summary

- `super()` function
    - problems with subclassing built-ins like `list`. Use `UserList` instead
    - Sometimes, it's easier to the appropriate ABC from `collections.abc` and write your own implementations
- Multiple Inheritance
    - `__mro__` addresses the problem of potential naming conflicts of inherited methods
    - `super()` can behave unexpectedly with multiple inheritance. It's designed to support mixin classes
    - We reviewed examples of multiple inheritance and mixins via Django, the python ABCs, and Tkinter.

Note that Go doesn't support inheritance - not even among interfaces. And it's doing just fine.

Perhaps the best advice about inheritance is: avoid it if you can. But often, we don't have a choice: the frameworks
we use impose their own design choices.